In [1]:
# !pip install langchain openai
# !pip install python-arango
# !pip install langchain-community
# !pip install langchain-openai
# !pip install --upgrade langchain langchain-community langchain-openai langgraph
# !pip install langgraph
# !pip install biomart

# !pip install DeepPurpose 
# !pip install torch torchvision torchaudio

# !pip install git+https://github.com/bp-kelley/descriptastorus
# !pip install pandas-flavor


In [46]:
import os
import sys
import requests
import ast
import json
from datetime import datetime
from glob import glob
from io import StringIO

import pandas as pd

from dotenv import load_dotenv
from arango import ArangoClient
from biomart import BiomartServer

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool

from DeepPurpose import utils
from DeepPurpose import DTI as models

from rdkit import Chem, DataStructs
from rdkit.Chem import MACCSkeys
from rdkit.Chem import Draw, AllChem

from Bio.PDB import MMCIFParser

In [23]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [24]:
db = ArangoClient(hosts="http://localhost:8529").db('NeuThera', username='root', password='openSesame')
arango_graph = ArangoGraph(db)

## Tooling

- **text_to_aql**: Executes a relevant AQL from user input
- **fetch_pdb_from_ensp**: Fetches PDB IDs from ENSP IDs
- **predict_binding_affinity**: uses DeepPurpose to find the binding affinity of a compound on a target protein
- 

In [25]:
@tool
def text_to_aql(query: str):
    """Execute a Natural Language Query in ArangoDB, and return the result as text."""
    
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    chain = ArangoGraphQAChain.from_llm(
        llm=llm,
        graph=arango_graph,  # Assuming arango_graph is already initialized
        verbose=True,
        allow_dangerous_requests=True
    )
    
    result = chain.invoke(query)

    return str(result["result"])

In [39]:
# @tool
def predict_binding_affinity(X_drug, X_target, y=[7.635]):
    """
    Predicts the binding affinity for given drug and target sequences.

    Parameters:
    X_drug (list): List containing the SMILES representation of the drug.
    X_target (list): List containing the amino acid sequence of the protein target.

    Returns:
    float: Predicted binding affinity (log(Kd) or log(Ki)).
    """

    print("Predicting binding affinity: ", X_drug, X_target)
    
    model = models.model_pretrained(path_dir='DTI_model')

    X_pred = utils.data_process(X_drug, X_target, y,
                                drug_encoding='CNN', 
                                target_encoding='CNN', 
                                split_method='no_split')
   
    predictions = model.predict(X_pred)

    return predictions[0]


In [28]:
@tool
def get_amino_acid_sequence_from_pdb(pdb_id):    
    """
    Extracts amino acid sequences from a given PDB structure file in CIF format.

    Args:
        pdb_id (str): pdb id of the protein.

    Returns:
        dict: A dictionary where keys are chain IDs and values are amino acid sequences.
    """

    print("Getting Amino Acid sequence for ", pdb_id)

    cif_file_path = f"./database/PDBlib/{pdb_id.lower()}.cif"

    parser = MMCIFParser(QUIET=True)
    structure = parser.get_structure("protein", cif_file_path)
    
    sequences = {}
    for model in structure:
        for chain in model:
            seq = "".join(residue.resname for residue in chain if residue.id[0] == " ")
            sequences[chain.id] = seq 
            
    return sequences

In [29]:
sys.path.append(os.path.abspath("./TamGen"))

In [30]:
from TamGen_custom import TamGenCustom

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

worker = TamGenCustom(
    data="./TamGen_Demo_Data",
    ckpt="checkpoints/crossdock_pdb_A10/checkpoint_best.pt",
    use_conditional=True
)

@tool
def prepare_pdb_data(pdb_id):
    """
    Checks if the PDB data for the given PDB ID is available.  
    If not, downloads and processes the data.

    ALWAYS RUN THIS FUNCTION BEFORE WORKING WITH PDB

    Args:
        pdb_id (str): PDB ID of the target structure.

    """

    DemoDataFolder="TamGen_Demo_Data"
    ligand_inchi=None
    thr=10

    out_split = pdb_id.lower()
    FF = glob(f"{DemoDataFolder}/*")
    for ff in FF:
        if f"gen_{out_split}" in ff:
            print(f"{pdb_id} is downloaded")
            return
    
    os.makedirs(DemoDataFolder, exist_ok=True)
    
    with open("tmp_pdb.csv", "w") as fw:
        if ligand_inchi is None:
            print("pdb_id", file=fw)
            print(f"{pdb_id}", file=fw)
        else:
            print("pdb_id,ligand_inchi", file=fw)
            print(f"{pdb_id},{ligand_inchi}", file=fw)

    script_path = os.path.abspath("TamGen/scripts/build_data/prepare_pdb_ids.py")
    os.system(f"python {script_path} tmp_pdb.csv gen_{out_split} -o {DemoDataFolder} -t {thr}")
    os.remove("tmp_pdb.csv")

@tool
def generate_compounds(pdb_id, num_samples=10, max_seed=30):
    """
    Generates and sorts compounds based on similarity to a reference molecule.

    Parameters:
    - pdb_id (str): The PDB ID of the target protein.
    - num_samples (int): Number of compounds to generate. (DEFAULT=500)
    - max_seed (int): Maximum seed variations. (DEFAULT=30)

    Returns:
    - dict: {
        'generated': [list of rdkit Mol objects],
        'reference': rdkit Mol object,
        'reference_smile': SMILE string of the reference compound
        'generated_smiles': [list of SMILES strings, sorted by similarity to reference]
      }
    """

    print("Generating Compounds for PDB ", pdb_id)
    try:
        # Ensure the required PDB data is prepared
        # prepare_pdb_data(pdb_id)

        worker.reload_data(subset=f"gen_{pdb_id.lower()}")

        print(f"Generating {num_samples} compounds...")
        generated_mols, reference_mol = worker.sample(
            m_sample=num_samples, 
            maxseed=max_seed
        )

        if reference_mol:
            # Ensure reference_mol is an RDKit Mol object
            if isinstance(reference_mol, str):
                reference_mol = Chem.MolFromSmiles(reference_mol)

            fp_ref = MACCSkeys.GenMACCSKeys(reference_mol)

            gens = []
            for mol in generated_mols:
                if isinstance(mol, str):  # Convert string SMILES to Mol
                    mol = Chem.MolFromSmiles(mol)
                if mol:  # Ensure conversion was successful
                    fp = MACCSkeys.GenMACCSKeys(mol)
                    similarity = DataStructs.FingerprintSimilarity(fp_ref, fp, metric=DataStructs.TanimotoSimilarity)
                    gens.append((mol, similarity))

            sorted_mols = [mol for mol, _ in sorted(gens, key=lambda e: e[1], reverse=True)]
        
        else:
            sorted_mols = generated_mols

        generated_smiles = [Chem.MolToSmiles(mol) for mol in sorted_mols if mol]

        reference_smile = Chem.MolToSmiles(reference_mol)

        return {
            "generated": sorted_mols,
            "reference": reference_mol,
            "reference_smile": reference_smile,
            "generated_smiles": generated_smiles
        }

    except Exception as e:
        print(f"Error in compound generation: {str(e)}")
        return {"error": str(e)}

Namespace(no_progress_bar=False, log_interval=1000, log_format=None, tensorboard_logdir='', tbmf_wrapper=False, seed=1, cpu=False, fp16=False, memory_efficient_fp16=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, min_loss_scale=0.0001, threshold_loss_scale=None, user_dir=None, criterion='cross_entropy', tokenizer=None, bpe=None, optimizer='nag', lr_scheduler='fixed', task='translation_coord', num_workers=1, skip_invalid_size_inputs_valid_test=False, max_tokens=1024, max_sentences=None, required_batch_size_multiple=8, dataset_impl=None, gen_subset='gen_8fln', num_shards=1, shard_id=0, path='checkpoints/crossdock_pdb_A10/checkpoint_best.pt', remove_bpe=None, quiet=False, model_overrides='{}', results_path=None, beam=20, nbest=20, max_len_a=0, max_len_b=200, min_len=1, match_source_len=False, no_early_stop=False, unnormalized=False, no_beamable_mm=False, lenpen=1, unkpen=0, replace_unk=None, sacrebleu=False, score_reference=False, prefix_size=0, prefix_string

In [49]:
@tool
def generate_report(columns, rows):
    """
    Generate a report in CSV format with a timestamped filename. This function uses pandas to create a CSV.
    
    Parameters:
    columns (list): List of column names.
    rows (list of lists): Data rows corresponding to the columns.
    
    Returns:
    str: Path of the generated CSV report.
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"report_{timestamp}.csv"
    
    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(filename, index=False)
    
    return filename

### Agentic RAG

In [50]:
tools = [ text_to_aql, get_amino_acid_sequence_from_pdb, prepare_pdb_data, generate_compounds, predict_binding_affinity, generate_report ]

def query_graph(query):
    
    query_template = f"""
    user input: {query}

    You are an advanced drug discovery assistant with multiple tools.

    - Use your tools as needed to assist in end-to-end drug discovery and answer user queries.
    - Always structure your output as valid JSON string so it can be parsed in python.
    - If possible, always try to generate reports for whatever output you get. Don't generate reports for errors
    - Do not add explanations or any extra text.

    !WARNING!
    YOU RUN FOR DEMO PURPOSES ONLY

    - DO NOT RUN TOOLS ESPECIALLY generate_compounds AND predict_binding_affinity MULTIPLE TIMES
    - IF MULTIPLE OUTPUTS ARE FOUND FOR PROCESSING, JUST PICK THE FIRST ONE AND RUN THE PROCEEDING FUNCTIONS WITH THAT.
    """
    
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
    app = create_react_agent(llm, tools)    
    final_state = app.invoke({"messages": [{"role": "user", "content": query_template}]})
    return final_state

In [51]:
output = query_graph("What proteins can you find me related to mitochondrial ribosomal protein L36? Generate some compounds and test their binding affinity")
# output = query_graph("Given pdb 5ool, generate some compounds")

output



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH gene, protein, gene-protein
FOR gene_doc IN gene
  FILTER gene_doc.gene_name == "mitochondrial ribosomal protein L36"
  FOR edge IN gene-protein
    FILTER edge._from == gene_doc._id
    FOR protein_doc IN protein
      FILTER protein_doc._id == edge._to
      RETURN protein_doc

AQL Query Execution Error: 
syntax error, unexpected - operator near '-protein
FOR gene_doc IN gene
  ...' at position 2:25

AQL Query (2):
WITH gene, protein, `gene-protein`
FOR gene_doc IN gene
  FILTER gene_doc.gene_name == "mitochondrial ribosomal protein L36"
  FOR edge IN `gene-protein`
    FILTER edge._from == gene_doc._id
    FOR protein_doc IN protein
      FILTER protein_doc._id == edge._to
      RETURN protein_doc

AQL Result:
[{'_key': '3j7y', '_id': 'protein/3j7y', '_rev': '_jUJ1i8u--s'}, {'_key': '3j9m', '_id': 'protein/3j9m', '_rev': '_jUJ1i8u--t'}, {'_key': '5ool', '_id': 'protein/5ool', '_rev': '_jUJ1i8u--u'}, {'_key': '6i9r', '_

  0%|          | 0/1 [00:00<?, ?it/s]

Processing 3j7y


100%|██████████| 1/1 [00:02<00:00,  2.20s/it]
Traceback (most recent call last):
  File "/Users/redomic/Documents/Projects/Hackathons/GDG/GDG-25/TamGen/scripts/build_data/prepare_pdb_ids.py", line 68, in <module>
    main()
  File "/Users/redomic/Documents/Projects/Hackathons/GDG/GDG-25/TamGen/scripts/build_data/prepare_pdb_ids.py", line 61, in main
    dump_data(all_data, args.name, output_dir=args.output,
  File "/Users/redomic/Documents/Projects/Hackathons/GDG/TamGen/fairseq/molecule_utils/external/fairseq_dataset_build_utils.py", line 947, in dump_data
    if processed_pairs[0].uniprot_id is not None:
       ~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


Generating Compounds for PDB  3j9m
Error in compound generation: Dataset not found: gen_3j9m (./TamGen_Demo_Data)


...

100%|██████████| 1/1 [00:03<00:00,  3.65s/it]
Traceback (most recent call last):
  File "/Users/redomic/Documents/Projects/Hackathons/GDG/GDG-25/TamGen/scripts/build_data/prepare_pdb_ids.py", line 68, in <module>
    main()
  File "/Users/redomic/Documents/Projects/Hackathons/GDG/GDG-25/TamGen/scripts/build_data/prepare_pdb_ids.py", line 61, in main
    dump_data(all_data, args.name, output_dir=args.output,
  File "/Users/redomic/Documents/Projects/Hackathons/GDG/TamGen/fairseq/molecule_utils/external/fairseq_dataset_build_utils.py", line 947, in dump_data
    if processed_pairs[0].uniprot_id is not None:
       ~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


Generating Compounds for PDB 5ool is downloaded
 5ool
| loaded 1 examples from: ./TamGen_Demo_Data/gen_5ool.tg-m1.tg
| loaded 1 examples from: ./TamGen_Demo_Data/gen_5ool.tg-m1.m1
| ./TamGen_Demo_Data gen_5ool tg-m1 1 examples
| loaded 1 examples from: ./TamGen_Demo_Data/gen_5ool.tg-m1.tg.coord
Generating 10 compounds...


 97%|█████████▋| 29/30 [04:57<00:10, 10.26s/it]


Getting Amino Acid sequence for  5ool
Predicting binding affinity:  ['CC(C)(COP(=O)(O)OP(=O)(O)OC[C@H]1O[C@@H](n2ccc(=N)nc2O)[C@H](O)[C@@H]1OP(=O)(O)O)[C@@H](O)C(O)=NCCC(O)=NCCS'] ['GCUAAACCUAGCCCCAAACCCCCACCUUACUACCAAACCUUAGCCAAACCAUUUACAUAAAGUAUAGGCGAUAGAAAUUGGGCGCAAUAGAUAUAGUACCGCAAGGGAAAGAUGAAAAAACCAAGCAUAAUAUAGCAAGGACUAACCCCUAUACCUUCUGCAUAAUGAAUUAACUAGAAAUAACUUUGCAAGGAGAGCCAAAGCUAAGACCCCCGAAACCAGACGAGCUACCUAAGAACAGCUAAGAGCACACCCGUCUAUGUAGCAAAAUAGUGGGAAGAUUUAUAGGUAGAGGCGACAAACCUACCGAGCCUGGUGAUAGCUGGUUGUCCAAGAUAGAAUCUUAGUUCAACUUUAAAUUUGCCCACAGAACCAAAUCCCCUUGUAAAUUUAACUGUUAGUCCAAAGAGGAACAGCUCUUUGGACACUAGGAAAAAACCUUGUAGAGAGAGUAAAAAAUUUAACACCCAUAGUAGGCCUAAAAGCAGCCACCAAUUAAGAAAGCGUUCAAGCUCAACACAAAAAUCCCAAACAUAUAACUGAACUCCUCACACCCAAUUGGACCAAUCUAUCACCCUAUAGAAGAACUAAUGUUAGUAUAAGUAACAUGAAAACAUUCUCCUCCGCAUAAGCCUGCGUCAGCAACUGACAAUUAACAGCCCAAUAUCUACAAUCAACCAACAAGUCAUUAUUACCCUCACUGUCAACCCAACACAGGCAUGCUCAUAAGGAAAGGUUAAAAAAAGUAAAAGGAACUCGGCAAAUCUUACCCCGCCUGUUUACCAAAAACAUCACCUCUAGCAUCACCAGUAUUAGAG

{'messages': [HumanMessage(content="\n    user input: What proteins can you find me related to mitochondrial ribosomal protein L36? Generate some compounds and test their binding affinity\n\n    You are an advanced drug discovery assistant with multiple tools.\n\n    - Use your tools as needed to assist in end-to-end drug discovery and answer user queries.\n    - Always structure your output as valid JSON string so it can be parsed in python.\n    - If possible, always try to generate reports for whatever output you get. Don't generate reports for errors\n    - Do not add explanations or any extra text.\n\n    !WARNING!\n    YOU RUN FOR DEMO PURPOSES ONLY\n\n    - DO NOT RUN TOOLS ESPECIALLY generate_compounds AND predict_binding_affinity MULTIPLE TIMES\n    - IF MULTIPLE OUTPUTS ARE FOUND FOR PROCESSING, JUST PICK THE FIRST ONE AND RUN THE PROCEEDING FUNCTIONS WITH THAT.\n    ", additional_kwargs={}, response_metadata={}, id='8a5ece78-2a28-4263-a1f0-ac5aeb6acfdd'),
  AIMessage(content

In [45]:
message = json.loads(output["messages"][-1].content)
message

{'error': 'Unable to generate compounds for the given PDB IDs due to missing datasets.'}